In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import time
import cv2

#set input, output folders
#set file name
name='H2226_SA28_WWW_SW6_PRE1'
input_folder=r'C:\Users\fr87_\Documents\Python code\Input folder'
file_name=name+'DLC_resnet50_event_arena2Nov17shuffle1_300000.csv'
vid_name=name+'.flv'
lab_vid_name=name+'DLC_resnet50_event_arena2Nov17shuffle1_300000_labeled.mp4'

output_folder=r'C:\Users\fr87_\Documents\Python code\Output folder'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

columns=  ['Time (s)','cap_x','cap_y','cap_likelihood','leftear_x','leftear_y','leftear_likelihood','rightear_x','rightear_y','rightear_likelihood','tailbase_x','tailbase_y','tailbase_likelihood']      
"""
Time isn't time now but whatever
"""
#this reads the DLC output, removes the text columns, converts to float and assigns the column names
df=pd.read_csv(os.path.join(input_folder, file_name), sep=',', header=1)

df.iloc[1:]

In [ ]:
df.columns=columns
df=df.iloc[1:]
df=df.reset_index(drop=True)
df=df.astype(float)

#stores original data and works on a copy
df2=df.copy()


In [ ]:
#cleanup the points by removing lowlikelihood points, and outside-the arena points. Points are converted to NaNs

def clean_up(dataframe, points, threshold, x_min, y_max):
    #points is a list of strings
    
    for item in points:
        dataframe['%s_x' %item].mask(dataframe['%s_likelihood' %item] < threshold, np.nan, inplace=True)
        dataframe['%s_x' %item].mask(dataframe['%s_x' %item] < x_min, np.nan, inplace=True)
        dataframe['%s_x' %item].mask(dataframe['%s_y' %item] > y_max, np.nan, inplace=True)
        print("Cleaned ", item, "_x")
        
        dataframe['%s_y' %item].mask(dataframe['%s_likelihood' %item] < threshold, np.nan, inplace=True)
        dataframe['%s_y' %item].mask(dataframe['%s_x' %item] < x_min, np.nan, inplace=True)
        dataframe['%s_y' %item].mask(dataframe['%s_y' %item] > y_max, np.nan, inplace=True)
        print("Cleaned ",item,"_y")
    
    return dataframe
            
my_points = ['cap', 'rightear', 'leftear', 'tailbase']

df2=clean_up(df2, my_points, 0.9, 250, 950)

In [ ]:
#Before and after cleaning graphs
"""
fig=plt.figure(figsize=(12,12))
ax = fig.add_subplot(2,2,1) 
ax.plot(df['cap_x'], df['cap_y'], c ='blue')
ax.scatter(df2['cap_x'], df2['cap_y'], c ='red',s=10)
ax.set_title("cap_x,y correction", fontsize=20)
ax.set_xlabel('cap_x', fontsize=15)
ax.set_ylabel('cap_y', fontsize=15)
ax.text( 100, 50, "Before", c='b', fontweight='bold', fontsize =10)
ax.text( 100, 0, "After", c='r', fontweight='bold', fontsize =10)

ax = fig.add_subplot(2,2,2) 
ax.plot(df['rightear_x'], df['rightear_y'], c ='orange')
ax.scatter(df2['rightear_x'], df2['rightear_y'], c ='red',s=10)
ax.set_title("rightear_x,y correction", fontsize=20)
ax.set_xlabel('rightear_x', fontsize=15)
ax.set_ylabel('rightear_y', fontsize=15)
ax.text( 100, 50, "Before", c='orange', fontweight='bold', fontsize =10)
ax.text( 100, 0, "After", c='r', fontweight='bold', fontsize =10)

ax = fig.add_subplot(2,2,3) 
ax.plot(df['leftear_x'], df['leftear_y'], c ='yellow')
ax.scatter(df2['leftear_x'], df2['rightear_y'], c ='red',s=10)
ax.set_title("leftear_x,y correction", fontsize=20)
ax.set_xlabel('leftear_x', fontsize=15)
ax.set_ylabel('leftear_y', fontsize=15)
ax.text( 100, 50, "Before", c='k', fontweight='bold', fontsize =10)
ax.text( 100, 0, "After", c='r', fontweight='bold', fontsize =10)

ax = fig.add_subplot(2,2,4) 
ax.plot(df['tailbase_x'], df['tailbase_y'], c ='g')
ax.scatter(df2['tailbase_x'], df2['tailbase_y'], c ='red',s=10)
ax.set_title("tailbase_x,y correction", fontsize=20)
ax.set_xlabel('tailbase_x', fontsize=15)
ax.set_ylabel('tailbase_y', fontsize=15)
ax.text( 100, 50, "Before", c='g', fontweight='bold', fontsize =10)
ax.text( 100, 0, "After", c='r', fontweight='bold', fontsize =10)

plt.show(block=False)
plt.close()
"""

In [ ]:
def edist_df(dataframe,p,q,row):
    #this function calculates the euclidean distance between selected points (p,q) e.g. 'cap' and 'leftear' for the selected row number 
    #the dataframe must contain p,q coordinates in the form 'p_x, 'p_y', 'q_x', 'q_y'
    #analogous to 
    #dataframe.apply(lambda x: dst.euclidean( [x['p_x'],x['p_y']], [x['q_x'],x['q_y']]), axis=1)

    p=[ dataframe['%s_x' %p].iloc[row] , dataframe['%s_y' %p].iloc[row] ]
    q=[ dataframe['%s_x' %q].iloc[row] , dataframe['%s_y' %q].iloc[row] ]
    value=math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))
    return value

def d_df(df,px,py,n):
    #calculates the euclidean distance between one frame and the previous for the poin p px and py are _x and _y coordinates 
    #row=n
    p=[ df[px].iloc[n] , df[py].iloc[n] ]
    q=[ df[px].iloc[n-1] , df[py].iloc[n-1] ]
    value=math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))
    return value
    
df2['d_cap']=df2['d_Rear']=df2['d_Lear']=df2['d_tail']=df2['cap_x']*0

for row in range(1,len(df2.index)):
    df2['d_cap'].iloc[row]= d_df(df2, 'cap_x','cap_y', row)
    df2['d_Rear'].iloc[row]= d_df(df2, 'rightear_x','rightear_y', row)
    df2['d_Lear'].iloc[row]= d_df(df2, 'leftear_x', 'leftear_y', row)
    df2['d_tail'].iloc[row]= d_df(df2, 'tailbase_x','tailbase_y', row)


In [ ]:
#plots cap summary graphs if necessary 
"""
fig=plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,2,1) 
ax.plot(df2['cap_x'], df2['cap_y'], color ='blue')
ax.scatter(df2[df2['d_cap']>50]['cap_x'], df2[df2['d_cap']>50]['cap_y'], c ='red', s=100)
ax.set_title('d>50 points for cap')

ax = fig.add_subplot(1,2,2) 
x1 = np.linspace(0.0, len(df2.index), len(df2.index))
ax.plot(x1, df2['d_cap'])
ax.set_title('d_cap')
plt.show(block=False)
plt.close()
"""

In [ ]:
#removes freames with d_cap>50 and anything inbetween if less than 10 frames apart, sets them to NaNs and interpolates cap coordinates
more=[]
frames_to_remove=[]
remove_me=df2[df2['d_cap']>50].index.tolist()

for i in list(range(len(remove_me)-1)):
    if remove_me[i+1]-remove_me[i] <= 10:
        k=1
        while remove_me[i]+k < remove_me[i+1]:
            more.append(remove_me[i]+k)
            k+=1
            frames_to_remove=remove_me+more
            
df2.loc[frames_to_remove, ['cap_x', 'cap_y']]=np.nan

df2[['cap_x', 'cap_y']]=df2[['cap_x', 'cap_y']].interpolate(method='linear')
for row in range(1,len(df2.index)):
    df2['d_cap'].iloc[row]= d_df(df2, 'cap_x','cap_y', row)
    
#at this stage, cap should be okay
"""
fig=plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,2,1) 
ax.plot(df2['cap_x'], df2['cap_y'], color ='blue')
ax.scatter(df2[df2['d_cap']>50]['cap_x'], df2[df2['d_cap']>50]['cap_y'], c ='red', s=100)
ax.set_title('d>50 points for cap')

ax = fig.add_subplot(1,2,2) 
x1 = np.linspace(0.0, len(df2.index), len(df2.index))
ax.plot(x1, df2['d_cap'])
ax.set_title('d_cap')
plt.show()
"""

if max(df2['d_cap'].dropna())>50:
    print('Alert: some points may still be incorrect')
else:
    print('Cap has converged')

In [ ]:
#calculates the distances of the points from cap:
df2[['cap_x', 'cap_y']]=df2[['cap_x', 'cap_y']].interpolate(method='linear')

df2['cap_Rear_distance']=df2['cap_Lear_distance']=df2['cap_tail_distance']=df['leftear_x']*0

for row in range(len(df2.index)):
    df2['cap_Lear_distance'].iloc[row]= edist_df(df2, 'leftear','cap', row)
    df2['cap_Rear_distance'].iloc[row]= edist_df(df2, 'rightear','cap', row)
    df2['cap_tail_distance'].iloc[row]= edist_df(df2, 'tailbase','cap', row)
    
#cleans ears and tail position with distance limit from cap 
#cap limit for ears=20
#cap limit for tail=50-150

R_before = df2['cap_Rear_distance'].copy()

df2['rightear_x'].mask(df2['cap_Rear_distance'] >20, np.nan, inplace=True)
df2['rightear_y'].mask(df2['cap_Rear_distance'] >20, np.nan, inplace=True)
df2[['rightear_x', 'rightear_y']]=df2[['rightear_x', 'rightear_y']].interpolate(method='linear')

L_before=df2['cap_Lear_distance'].copy()

df2['leftear_x'].mask(df2['cap_Lear_distance'] >20, np.nan, inplace=True)
df2['leftear_y'].mask(df2['cap_Lear_distance'] >20, np.nan, inplace=True)
df2[['leftear_x', 'leftear_y']]=df2[['leftear_x', 'leftear_y']].interpolate(method='linear')

tail_before=df2['cap_tail_distance'].copy()
    
df2['tailbase_x'].mask((df2['cap_tail_distance'] >150)|(df2['cap_tail_distance'] <50), np.nan, inplace=True)
df2['tailbase_y'].mask((df2['cap_tail_distance'] >150)|(df2['cap_tail_distance'] <50), np.nan, inplace=True)
df2[['tailbase_x', 'tailbase_y']]=df2[['tailbase_x', 'tailbase_y']].interpolate(method='linear')

#recalculates cap-point distances for ears and d_tail, d(R/L)ear
for row in range(len(df2.index)):
    df2['cap_Rear_distance'].iloc[row]= edist_df(df2, 'rightear', 'cap', row)
    df2['cap_Lear_distance'].iloc[row]= edist_df(df2, 'leftear','cap', row)
    df2['d_Rear'].iloc[row]= d_df(df2, 'rightear_x','rightear_y', row)
    df2['d_Lear'].iloc[row]= d_df(df2, 'leftear_x', 'leftear_y', row)
    df2['d_tail'].iloc[row]= d_df(df2, 'tailbase_x','tailbase_y', row)


#further cleaning for tail: removes points with too high deviations from frame to frame

df2['tailbase_x'].mask((df2['d_tail'] >30), np.nan, inplace=True)
df2['tailbase_y'].mask((df2['d_tail'] >30), np.nan, inplace=True)
df2[['tailbase_x', 'tailbase_y']]=df2[['tailbase_x', 'tailbase_y']].interpolate(method='linear')

            
df2.loc[frames_to_remove, ['cap_x', 'cap_y']]=np.nan

df2[['cap_x', 'cap_y']]=df2[['cap_x', 'cap_y']].interpolate(method='linear')
for row in range(1,len(df2.index)):
    df2['d_cap'].iloc[row]= d_df(df2, 'cap_x','cap_y', row)

for row in range(len(df2.index)):
    df2['cap_tail_distance'].iloc[row]= edist_df(df2, 'tailbase','cap', row)
    df2['d_tail'].iloc[row]= d_df(df2, 'tailbase_x','tailbase_y', row)



In [ ]:
fig=plt.figure(figsize=(16,5))
ax=plt.subplot2grid((4, 1), (0, 0), rowspan=1)
ax.plot(R_before, color='k',linewidth=5)
ax.plot(df2['cap_Rear_distance'], color='r')
ax.set_ylabel('Cap-Right ear distance')
ax.text( 200, 25, "before", c='k', fontweight='bold', fontsize =10)
ax.text( 200, 20, "after", c='r', fontweight='bold', fontsize =10)
ax.set_ylim(top=30)

ax=plt.subplot2grid((4, 1), (1, 0), rowspan=1)
ax.plot(L_before, color='k',linewidth=5)
ax.plot(df2['cap_Lear_distance'], color='orange')
ax.set_ylabel('Cap-Left ear distance')
ax.text( 200, 25, "before", c='k', fontweight='bold', fontsize =10)
ax.text( 200, 20, "after", c='orange', fontweight='bold', fontsize =10)
ax.set_ylim(top=30)

ax=plt.subplot2grid((4, 1), (2, 0), rowspan=2)
ax.plot(tail_before, linewidth=5, color='k')
ax.plot(df2['cap_tail_distance'], color='green')
ax.set_ylabel('Cap-tail distance')
ax.text( 200, 250, "before", c='k', fontweight='bold', fontsize =10)
ax.text( 200, 210, "after", c='green', fontweight='bold', fontsize =10)
ax.set_ylim(top=300)
plt.show(block=False)
plt.close()

In [ ]:
fig=plt.figure(figsize=(16,7))
ax=fig.add_subplot(1,2,1)
ax.plot(df2['cap_x'], df2['cap_y'], color ='blue')
ax.plot(df2['tailbase_x'], df2['tailbase_y'], color ='green')
ax.scatter(df2[df2['d_tail']>30]['tailbase_x'], df2[df2['d_tail']>30]['tailbase_y'], c ='orange', s=150)
ax.set_title('cap vs tail')
ax=fig.add_subplot(1,2,2)
ax.plot(df2['d_tail'], color='black')
ax.set_title('d_tail');
plt.show(block=False)
plt.close()

In [ ]:
fig=plt.figure(figsize=(16,5))
ax = fig.add_subplot(1,3,1) 
ax.plot(df2['cap_x'], df2['cap_y'], c ='blue')
ax.set_title("Cap vs right ear", fontsize=20)

ax.text( 100, 150, "Cap", c='b', fontweight='bold', fontsize =10)
ax.text( 100, 100, "Right ear", c='r', fontweight='bold', fontsize =10)
ax.text( 100, 50, "Left ear", c='orange', fontweight='bold', fontsize =10)
ax.text( 100, 0, "tail base", c='g', fontweight='bold', fontsize =10)
ax.plot(df2['rightear_x'], df2['rightear_y'], c ='r')


ax = fig.add_subplot(1,3,2) 
ax.plot(df2['cap_x'], df2['cap_y'], c ='blue')
ax.set_title("Cap vs left ear", fontsize=20)
ax.plot(df2['leftear_x'], df2['leftear_y'], c ='orange')

ax = fig.add_subplot(1,3,3) 
ax.plot(df2['cap_x'], df2['cap_y'], c ='blue')
ax.set_title("Cap vs tail base", fontsize=20)
ax.plot(df2['tailbase_x'], df2['tailbase_y'], c ='g')

plt.show(block=False)
plt.close()

In [ ]:
# export video for checking from the orginal flv
df3=df2.fillna(0)

FPS = 25

cap = cv2.VideoCapture(os.path.join(input_folder, vid_name))
width=cap.get(3)
height=cap.get(4)
fourcc=cv2.VideoWriter_fourcc(*'mp4v')

video=cv2.VideoWriter(os.path.join(output_folder,  vid_name[:-4]+"_DLC_labelled.mp4"), fourcc, int(FPS), (int(width), int(height)))

i=0

for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1):
    ret, frame = cap.read()
    newer = cv2.circle(frame, (int(df3['tailbase_x'][i]), int(df3['tailbase_y'][i])), 10, (0, 0, 255), -1)    
    newer = cv2.circle(newer, (int(df3['cap_x'][i]), int(df3['cap_y'][i])), 10, (255, 0, 0), -1)
    newer = cv2.circle(newer, (int(df3['leftear_x'][i]), int(df3['leftear_y'][i])), 6, (255, 225, 0), -1)
    newer = cv2.circle(newer, (int(df3['rightear_x'][i]), int(df3['rightear_y'][i])), 6, (0, 225, 255), -1)


    video.write(newer)

    
video.release()
cap.release()

if os.path.exists(os.path.join(output_folder,vid_name[:-4]+"_DLC_labelled.mp4")):
    print('Video saved successfully')
    

In [ ]:
df2

In [ ]:
cap = cv2.VideoCapture(os.path.join(input_folder, vid_name))

int(cap.get(cv2.CAP_PROP_FRAME_COUNT))